In [1]:
from lexikos import Lexicon
lexicon = Lexicon()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

df = pd.read_csv("../lexikos/dict/cmudict-ipa/librispeech-lexicon-200k-allothers-g2p-ipa.tsv", sep="\t", names=["word", "phoneme"])
df["phoneme"] = df["phoneme"].apply(lexicon._normalize_phonemes)
df = df[df["phoneme"].apply(lambda x: len(x.split()) > 1)].sort_values(by=['word'])
df["phoneme"] = df["phoneme"].apply(lambda x: x.replace(" . ", " "))

In [6]:
phonemes = set([p for phoneme in df["phoneme"] for p in phoneme.split()])

In [7]:
mapping = {
    "v": "f",
    "ð": "t",
    "ɝ": "ə ɹ",
    "θ": "t",
}

In [8]:
def transliterate(phonemes):
    for k, v in mapping.items():
        phonemes = phonemes.replace(k, v)
    return phonemes

In [9]:
df["phoneme_id"] = df["phoneme"].apply(transliterate)

In [10]:
df = df[df["phoneme"] != df["phoneme_id"]]
df = df.drop(["phoneme"], axis=1).reset_index(drop=True)
df

,word,phoneme_id
0,a'ter,eɪ t ə ɹ
1,a'ternoon,eɪ t ə ɹ n u n
2,a'terwards,eɪ t ə ɹ w ə ɹ d z
3,a'thegither,eɪ t ə dʒ ɪ t ə ɹ
4,a'thing,eɪ t ɪ ŋ
...,...,...
59463,zwemer,z w i m ə ɹ
59464,zwengler,z w ɪ ŋ ə l ə ɹ
59465,zwillievic,z w ɪ l j ə f ɪ k
59466,zwinger,z w ɪ ŋ ə ɹ


In [ ]:
df.to_csv("../lexikos/dict/synthetic/librispeech-lexicon-en-id.tsv", sep="\t", index=False, header=None)